<a href="https://colab.research.google.com/github/bharadwaj-vedula/Direct-Marketing-Optimisation/blob/main/Rebel_Foods_Final_Prediction_%26_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [131]:
! pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 42.1 MB/s eta 0:00:00


In [103]:
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

In [143]:
import os
import pandas as pd
import numpy as np
from collections import Counter

import pulp

import seaborn as sns
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

# Paths

In [105]:
class config:
  path = '/content/drive/MyDrive/Rebel Foods Assignment/DataScientist_CaseStudy_Dataset  (1) (1).xlsx'
  mf_classifier = '/content/drive/MyDrive/Rebel Foods Assignment/Models/XGBClassifier_MF.json'
  mf_regressor = '/content/drive/MyDrive/Rebel Foods Assignment/Models/XGBRegressor_MF.json'
  cc_classifier = '/content/drive/MyDrive/Rebel Foods Assignment/Models/XGBClassifier_CC.json'
  cc_regressor = '/content/drive/MyDrive/Rebel Foods Assignment/Models/XGBRegressor_CC.json'
  cl_classifier = '/content/drive/MyDrive/Rebel Foods Assignment/Models/XGBClassifier_CL.json'
  cl_regressor =  '/content/drive/MyDrive/Rebel Foods Assignment/Models/XGBRegressor_CL.json'


In [106]:
df_dem = pd.read_excel(config.path,sheet_name = 'Soc_Dem')
df_actBalance = pd.read_excel(config.path,sheet_name = 'Products_ActBalance')
df_inflowOutflow = pd.read_excel(config.path, sheet_name = 'Inflow_Outflow')
df_salesRevenue = pd.read_excel(config.path,sheet_name = 'Sales_Revenues')

In [107]:
sales_client = df_salesRevenue['Client'].values

In [108]:
df = df_dem.merge(df_actBalance, how= 'inner',on='Client').merge(df_inflowOutflow,how= 'inner',on='Client')

In [109]:
df

,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,ActBal_CA,ActBal_SA,ActBal_MF,ActBal_OVD,ActBal_CC,ActBal_CL,VolumeCred,VolumeCred_CA,TransactionsCred,TransactionsCred_CA,VolumeDeb,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder
0,909,M,21,27,1,NaN,NaN,1.0,NaN,1.0,4.710714,NaN,NaN,0.0,NaN,4291.996429,789.129643,738.230000,4,3,450.678571,448.892857,178.571429,0.000000,166.571429,8,7,1,0,4
1,1217,M,38,165,1,NaN,NaN,NaN,NaN,NaN,6752.244643,NaN,NaN,NaN,NaN,NaN,0.002143,0.002143,1,1,714.285714,714.285714,0.000000,0.000000,714.285714,1,1,0,0,1
2,850,F,49,44,1,NaN,NaN,NaN,NaN,NaN,43.523214,NaN,NaN,NaN,NaN,NaN,1392.402857,1392.402857,3,3,1226.345357,1226.345357,0.000000,0.000000,121.928571,6,6,0,0,1
3,1473,M,54,34,1,1.0,NaN,NaN,1.0,1.0,29.024286,14447.801429,NaN,NaN,653.91,1132.590357,1787.127500,939.128929,14,5,3875.137857,3794.580714,357.142857,444.597500,2076.785714,48,38,1,26,11
4,1038,M,29,106,1,NaN,NaN,NaN,NaN,NaN,27.035714,NaN,NaN,NaN,NaN,NaN,0.006071,0.006071,1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1582,409,M,31,91,1,NaN,NaN,1.0,NaN,NaN,348.402857,NaN,NaN,0.0,NaN,NaN,469.179643,469.179643,3,3,465.092857,465.092857,178.571429,11.414286,271.714286,12,12,1,1,9
1583,384,M,23,160,1,NaN,NaN,NaN,NaN,NaN,2418.876786,NaN,NaN,NaN,NaN,NaN,87.500357,87.500357,2,2,88.443929,88.443929,50.000000,37.372500,1.071429,8,8,2,5,1
1584,977,M,46,59,1,NaN,NaN,NaN,NaN,NaN,2639.308571,NaN,NaN,NaN,NaN,NaN,71.428571,71.428571,1,1,76.103571,76.103571,75.000000,1.067857,0.035714,4,4,2,1,1
1585,629,F,61,173,1,NaN,2.0,NaN,NaN,NaN,61.766786,NaN,34387.583571,NaN,NaN,NaN,1064.190000,1064.190000,3,3,817.462143,817.462143,660.714286,41.355000,115.178571,17,17,3,5,8


In [110]:
df = df.loc[(~df['Client'].isin(sales_client))]

In [111]:
sexMapping = {'F':np.int16(0),'M':np.int16(1)}
df['Sex'] = df['Sex'].map(sexMapping)
df.fillna(0,inplace= True)

In [113]:
df.reset_index(inplace = True, drop= True)

In [114]:
df

,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,ActBal_CA,ActBal_SA,ActBal_MF,ActBal_OVD,ActBal_CC,ActBal_CL,VolumeCred,VolumeCred_CA,TransactionsCred,TransactionsCred_CA,VolumeDeb,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder
0,909,1.0,21,27,1,0.0,0.0,1.0,0.0,1.0,4.710714,0.0,0.000000,0.000000,0.0,4291.996429,789.129643,738.230000,4,3,450.678571,448.892857,178.571429,0.000000,166.571429,8,7,1,0,4
1,699,1.0,37,175,1,0.0,4.0,1.0,0.0,0.0,1823.057143,0.0,18491.444286,0.000000,0.0,0.000000,1033.496071,778.370000,8,6,661.483214,566.126071,89.285714,0.000000,216.892857,13,10,2,0,5
2,528,0.0,19,70,1,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,435.682143,435.682143,2,2,390.056429,390.056429,125.000000,70.842143,190.821429,10,10,4,3,2
3,1145,1.0,61,45,1,0.0,0.0,0.0,0.0,0.0,324.710000,0.0,0.000000,0.000000,0.0,0.000000,132.158929,132.158929,3,3,3.392857,3.392857,0.000000,0.000000,0.000000,1,1,0,0,0
4,517,0.0,41,28,1,0.0,0.0,0.0,0.0,0.0,17.051071,0.0,0.000000,0.000000,0.0,0.000000,570.157857,570.157857,2,2,557.896429,557.896429,71.428571,224.146429,251.178571,17,17,2,10,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,419,0.0,33,82,1,0.0,0.0,1.0,0.0,0.0,352.782143,0.0,0.000000,0.000000,0.0,0.000000,1406.490714,934.384643,7,5,1404.483571,934.384643,0.000000,0.000000,0.000000,9,5,0,0,0
632,1024,1.0,65,13,1,0.0,0.0,0.0,0.0,0.0,193.040000,0.0,0.000000,0.000000,0.0,0.000000,178.579643,178.579643,2,2,109.107143,109.107143,107.142857,0.000000,0.000000,3,3,2,0,0
633,464,1.0,68,153,2,0.0,0.0,1.0,0.0,0.0,2249.049643,0.0,0.000000,237.297143,0.0,0.000000,811.332857,811.332857,3,3,577.385714,577.385714,160.714286,65.421429,347.857143,12,12,2,3,6
634,409,1.0,31,91,1,0.0,0.0,1.0,0.0,0.0,348.402857,0.0,0.000000,0.000000,0.0,0.000000,469.179643,469.179643,3,3,465.092857,465.092857,178.571429,11.414286,271.714286,12,12,1,1,9


# Model Predictions

In [116]:
# P(Accepts Mutual Fund)
Classifier_MF = XGBClassifier()
Classifier_MF.load_model(config.mf_classifier)
y_preds = Classifier_MF.predict_proba(df.iloc[:,1:])

p_mf = []

for pred in y_preds:
  p_mf.append(pred[1])

# P(Accepts Credit Card)
Classifier_CC= XGBClassifier()
Classifier_CC.load_model(config.cc_classifier)
y_preds = Classifier_CC.predict_proba(df.iloc[:,1:])

p_cc = []

for pred in y_preds:
  p_cc.append(pred[1])

# P(Accepts Consumer Loan)
Classifier_CL= XGBClassifier()
Classifier_CL.load_model(config.cl_classifier)
y_preds = Classifier_CL.predict_proba(df.iloc[:,1:])

p_cl = []

for pred in y_preds:
  p_cl.append(pred[1])


In [120]:
Regression_MF = XGBRegressor()
Regression_MF.load_model(config.mf_regressor)
y_preds = Regression_MF.predict(df.iloc[:,1:])
e_mf = np.expm1(y_preds)

Regression_CC = XGBRegressor()
Regression_CC.load_model(config.cc_regressor)
y_preds = Regression_CC.predict(df.iloc[:,1:])
e_cc = np.expm1(y_preds)


Regression_CL = XGBRegressor()
Regression_CL.load_model(config.cl_regressor)
y_preds = Regression_CL.predict(df.iloc[:,1:])
e_cl = np.expm1(y_preds)



In [137]:
df_lp = pd.DataFrame({
      'Client' : df['Client'],
      'P(MF)' : p_mf,
      'P(CC)' : p_cc,
      'P(CL)' : p_cl,
      'P(Revenue|MF)' : e_mf,
      'P(Revenue|CC)' : e_cc,
      'P(Revenue|CL)' : e_cl,

})

In [138]:
df_lp['E(Revenue|MF)'] = df_lp['P(MF)'] * df_lp['P(Revenue|MF)']
df_lp['E(Revenue|CC)'] = df_lp['P(CC)'] * df_lp['P(Revenue|CC)']
df_lp['E(Revenue|CL)'] = df_lp['P(CL)'] * df_lp['P(Revenue|CL)']


In [141]:
df_lp.loc[df_lp['P(Revenue|CL)'] < df_lp['P(Revenue|MF)']]

,Client,P(MF),P(CC),P(CL),P(Revenue|MF),P(Revenue|CC),P(Revenue|CL),E(Revenue|MF),E(Revenue|CC),E(Revenue|CL)


In [150]:
df_lp.index = df_lp['Client']

# PuLP

In [152]:
df_lp.loc[909,]

,909
Client,909.000000
P(MF),0.199086
P(CC),0.422901
P(CL),0.510219
P(Revenue|MF),2.586280
P(Revenue|CC),3.227037
P(Revenue|CL),9.260349
E(Revenue|MF),0.514893
E(Revenue|CC),1.364716
E(Revenue|CL),4.724803


In [165]:
prob = pulp.LpProblem("Direct_Marketing_Optimization", pulp.LpMaximize)

decisions = [(i, j) for i in df_lp['Client'] for j in df_lp.columns[-3:]]

x = pulp.LpVariable.dicts("Offer", decisions, cat='Binary')

prob += pulp.lpSum(
    df_lp.loc[i, j] * x[(i, j)] for i, j in decisions
), "Total_Expected_Revenue"

# Constraint 1: Total number of clients contacted cannot exceed 100
prob += pulp.lpSum(x[(i, j)] for i, j in decisions) <= 100, "Total_Contacts_Limit"

# Constraint 2: Each client can be offered at most one product
for client in df_lp.index:
    prob += pulp.lpSum(x[(client, j)] for j in df_lp.columns[-3:]) <= 1, f"Single_Offer_for_{client}"

# Constraint 3: minimum 20% should be credit card targeting
prob += pulp.lpSum(x[(client, 'E(Revenue|CC)')] for client in df_lp.index) >= 20

# Constraint 4: minimum 15% should be Mutual Fund users
prob += pulp.lpSum(x[(client, 'E(Revenue|MF)')] for client in df_lp.index) >= 15


# Solve the LP problem
prob.solve()


1

In [170]:
total_revenue = 0
contact_count = 0
contact_plan = []

print("--- Optimal Contact Plan ---")
for i, j in decisions:
    if x[(i, j)].varValue == 1:
        score = df_lp.loc[i, j]
        contact_plan.append({'Client': i, 'Product': j, 'Oppurtunity ExpectedRevenue': score})
        total_revenue += score
        contact_count += 1

plan_df = pd.DataFrame(contact_plan).sort_values(by='Oppurtunity ExpectedRevenue', ascending=False).reset_index(drop=True)


--- Optimal Contact Plan ---


In [171]:
plan_df

,Client,Product,Oppurtunity ExpectedRevenue
0,485,E(Revenue|CL),7.455493
1,878,E(Revenue|CL),7.339280
2,731,E(Revenue|CL),7.174396
3,498,E(Revenue|CL),7.171899
4,562,E(Revenue|CL),6.953811
...,...,...,...
95,958,E(Revenue|MF),2.180779
96,910,E(Revenue|MF),2.168664
97,1614,E(Revenue|MF),2.159476
98,1047,E(Revenue|MF),2.111937


In [188]:
df_lp

,Client,P(MF),P(CC),P(CL),P(Revenue|MF),P(Revenue|CC),P(Revenue|CL),E(Revenue|MF),E(Revenue|CC),E(Revenue|CL)
Client,,,,,,,,,,
909,909,0.199086,0.422901,0.510219,2.586280,3.227037,9.260349,0.514893,1.364716,4.724803
699,699,0.437693,0.511148,0.617224,2.651072,3.337512,10.529661,1.160356,1.705961,6.499165
528,528,0.089096,0.440245,0.567890,2.523397,3.038247,9.612134,0.224824,1.337573,5.458633
1145,1145,0.331829,0.436007,0.374665,2.541119,4.533344,9.440073,0.843216,1.976569,3.536868
517,517,0.572752,0.452796,0.475451,2.795416,3.999187,10.418554,1.601079,1.810815,4.953514
...,...,...,...,...,...,...,...,...,...,...
419,419,0.312736,0.432711,0.461836,2.886852,3.490295,9.890056,0.902822,1.510288,4.567582
1024,1024,0.133226,0.457628,0.355773,2.541119,3.429219,10.418554,0.338544,1.569308,3.706641
464,464,0.144879,0.489747,0.475695,2.766126,3.936384,10.591425,0.400753,1.927831,5.038293


In [200]:
revenues = []
for i,j in zip(plan_df.iloc[:,0].values,plan_df.iloc[:,1].values):

  if j == "E(Revenue|CL)":
    revenues.append(df_lp.loc[i,"P(Revenue|CL)"])
  elif j == "E(Revenue|MF)":
    revenues.append(df_lp.loc[i,"P(Revenue|MF)"])
  elif j == "E(Revenue|CC)":
    revenues.append(df_lp.loc[i,"P(Revenue|CC)"])


In [202]:
plan_df['Predicted Revenues'] =   revenues

In [212]:
plan_df.groupby(['Product']).count()

,Client,Oppurtunity ExpectedRevenue,Predicted Revenues
Product,,,
E(Revenue|CC),20,20,20
E(Revenue|CL),65,65,65
E(Revenue|MF),15,15,15


In [220]:
plan_df['Predicted Revenues'].sum()

np.float32(766.98)

In [217]:
aggregated_df = plan_df.groupby('Product').agg(
        client=('Client', 'count'),  # Sum of 'Value1', new column named 'sum_value1'
        Predicted_revenues=('Predicted Revenues', 'sum'),
    )

In [218]:
aggregated_df

,client,Predicted_revenues
Product,,
E(Revenue|CC),20,83.309937
E(Revenue|CL),65,642.699463
E(Revenue|MF),15,40.970592


In [221]:
plan_df.to_csv('/content/drive/MyDrive/Rebel Foods Assignment/Contact Plan.csv',index = False)